# OliveYoung 상품 정보 스크래핑

In [1]:
# import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 검색 대상
search_word = "립밤"

# today's date
today = time.strftime("%y-%m-%d", time.localtime())

In [2]:
# target url format
url_olive = "http://www.oliveyoung.co.kr/store/search/getSearchMain.do"

In [3]:
# 해당 키워드 검색시 마지막 페이지 숫자 반환
def extract_olive_pages():
    r = requests.get(f"{url_olive}?startCount=0&query={search_word}")
    soup = BeautifulSoup(r.text, 'html.parser')
    pagination = soup.find('div', class_='pageing')
    links = pagination.find_all('a')
    pages = []
    for link in links:
        pages.append(int(link.string))
    last_page = pages[-1]
    return last_page

In [4]:
# 현재 페이지에서 상품별 정보 추출 및 반환
def extract_prd(html):
    img_link = html.select_one("a > img[src]")['src']
    brand = html.select_one(".prd_name > a > .tx_brand").string
    name = html.select_one(".prd_name > a > .tx_name").string
    if html.select_one(".prd_price > .tx_org") is None:
        org_price = html.select_one(".prd_price > .tx_cur > .tx_num").string
    else:
        org_price = html.select_one(".prd_price > .tx_org > .tx_num").string
    cur_price = html.select_one(".prd_price > .tx_cur > .tx_num").string
    date = today 
    return {
        "img_link": img_link,
        "brand": brand,
        "name": name,
        "org_price": org_price,
        "cur_price": cur_price,
        "date": today
    }

In [5]:
# 첫번째 페이지부터 마지막 페이지까지 상품 정보 스크래핑
def extract_olive_prds(last_page):
    prds = []
    for page in range(last_page):
        time.sleep(1)
        r = requests.get(f"{url_olive}?startCount={page * 24}&query={search_word}")
        soup = BeautifulSoup(r.text, "html.parser")
        prd_infos = soup.find_all("div", class_="prd_info")
        for prd_info in prd_infos:
            prd = extract_prd(prd_info)
            prds.append(prd)
    return prds

In [6]:
# 해당 검색어의 마지막 페이지 수 저장
last_page = extract_olive_pages()

# 스크랩핑 결과를 DataFrame으로 변환 및 컬럼 정렬
prd_df = pd.DataFrame(extract_olive_prds(last_page), columns=["date", "brand", "name", "org_price", "cur_price", "img_link"])

# 결과 확인
prd_df

,date,brand,name,org_price,cur_price,img_link
0,19-12-07,유리아쥬,[어워즈한정기획] 유리아쥬 스틱레브르 오리지널 4G 듀오 기획(보습 립밤),"18,000","16,000",http://image.oliveyoung.co.kr/uploads/images/g...
1,19-12-07,버츠비,[어워즈/한정기획] 버츠비 자몽 립밤 어워즈 한정기획(1912),"6,000","6,000",http://image.oliveyoung.co.kr/uploads/images/g...
2,19-12-07,바이오더마,바이오더마 아토덤 립스틱 1+1,"15,000","12,000",http://image.oliveyoung.co.kr/uploads/images/g...
3,19-12-07,아이소이,[온라인단독]아이소이 불가리안 로즈 립 트리트먼트 밤(퓨어레드),"19,800","15,800",http://image.oliveyoung.co.kr/uploads/images/g...
4,19-12-07,버츠비,버츠비 틴티드 립밤 6 colors,"12,000","7,800",http://image.oliveyoung.co.kr/uploads/images/g...
5,19-12-07,히말라야,히말라야 립밤 10g,"3,500","3,500",http://image.oliveyoung.co.kr/uploads/images/g...
6,19-12-07,버츠비,버츠비 로즈마리 레몬 핸드크림 &자몽 립밤 기프트 세트,"11,800","8,900",http://image.oliveyoung.co.kr/uploads/images/g...
7,19-12-07,버츠비,버츠비 립밤 트리오 기프트 세트,"14,400","9,900",http://image.oliveyoung.co.kr/uploads/images/g...
8,19-12-07,닥터브로너스,[닥터브로너스]베이비마일드 오가닉 립밤 4g,"5,900","5,300",http://image.oliveyoung.co.kr/uploads/images/g...
9,19-12-07,카멕스,카멕스 모이스처라이징 립밤 클래식 튜브,"4,000","3,600",http://image.oliveyoung.co.kr/uploads/images/g...
